# Laufzeitanalyse: SHAP vs SHAPIQ

Dieses Notebook vergleicht verschiedene Approximationsverfahren zur Berechnung von Shapley-Werten.

# Ziel
Analyse von:
- Laufzeit (x-Achse)
- Approximationsgenauigkeit (L1 & L2 Fehler) gegenüber Referenzwerten

# Verglichene Methoden
| Bibliothek | Methode                  |
|------------|--------------------------|
| `shapiq`   | KernelSHAP, SVARM, PermutationSamplingSV |
| `shap`     | KernelExplainer, PermutationExplainer    |

# Datensätze
- Bike Sharing
- California Housing

# Modelle
- Lineare Regression
- Random Forest Regressor

Imports & Einstellungen

In [1]:
import shapiq
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import shap
from shapiq import TabularExplainer
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 7)

import random
np.random.seed(42)
random.seed(42)



Daten vorbereiten & subsample Background (für schnellere Laufzeiten)

In [2]:
# --- Daten laden und vorverarbeiten ---

X_bike, y_bike = shapiq.datasets.load_bike_sharing()
X_cal, y_cal = shapiq.datasets.load_california_housing()

print("Bike Sharing - X shape:", X_bike.shape, "y shape:", y_bike.shape)
print("California Housing - X shape:", X_cal.shape, "y shape:", y_cal.shape)

def preprocess_data(X, y, categorical_cols=None, sample_size=None):
    if isinstance(X, pd.DataFrame):
        if categorical_cols is not None:
            X_processed = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
        else:
            cat_cols = X.select_dtypes(include=['category', 'object']).columns.tolist()
            X_processed = pd.get_dummies(X, columns=cat_cols, drop_first=True)
    else:
        X_processed = pd.DataFrame(X)
    
    y_processed = pd.Series(y) if not isinstance(y, pd.Series) else y
    
    if sample_size is not None and len(X_processed) > sample_size:
        sampled_indices = X_processed.sample(n=sample_size, random_state=42).index
        X_processed = X_processed.loc[sampled_indices].reset_index(drop=True)
        y_processed = y_processed.loc[sampled_indices].reset_index(drop=True)
    else:
        X_processed = X_processed.reset_index(drop=True)
        y_processed = y_processed.reset_index(drop=True)
    
    return X_processed, y_processed

bike_categorical_cols = ['season', 'weather']

X_bike_proc, y_bike_proc = preprocess_data(X_bike, y_bike, categorical_cols=bike_categorical_cols)
X_cal_proc, y_cal_proc = preprocess_data(X_cal, y_cal)

sample_fraction = 0.1

X_bike_proc_sampled = X_bike_proc.sample(frac=sample_fraction, random_state=42)
y_bike_proc_sampled = y_bike_proc.loc[X_bike_proc_sampled.index]

X_cal_proc_sampled = X_cal_proc.sample(frac=sample_fraction, random_state=42)
y_cal_proc_sampled = y_cal_proc.loc[X_cal_proc_sampled.index]

print("Sampled preprocessed Bike Sharing shape:", X_bike_proc_sampled.shape)
print("Sampled preprocessed California Housing shape:", X_cal_proc_sampled.shape)

Bike Sharing - X shape: (17379, 12) y shape: (17379,)
California Housing - X shape: (20640, 8) y shape: (20640,)
Sampled preprocessed Bike Sharing shape: (1738, 16)
Sampled preprocessed California Housing shape: (2064, 8)


Modelle trainieren

In [3]:
# --- Train-Test Split ---

Xb_train, Xb_test, yb_train, yb_test = train_test_split(
    X_bike_proc_sampled, y_bike_proc_sampled, test_size=0.2, random_state=42
)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cal_proc_sampled, y_cal_proc_sampled, test_size=0.2, random_state=42
)

# --- Modelle trainieren ---

model_bike_rf = RandomForestRegressor(random_state=42)
model_bike_rf.fit(Xb_train, yb_train)
print("Bike Sharing RandomForestRegressor trainiert.")

model_cal_rf = RandomForestRegressor(random_state=42)
model_cal_rf.fit(Xc_train, yc_train)
print("California Housing RandomForestRegressor trainiert.")

model_bike_lr = LinearRegression()
model_bike_lr.fit(Xb_train, yb_train)
print("Bike Sharing LinearRegression trainiert.")

model_cal_lr = LinearRegression()
model_cal_lr.fit(Xc_train, yc_train)
print("California Housing LinearRegression trainiert.")

# --- Wrapper Funktion für sauberes predict mit Feature-Namen ---

def model_predict_wrapper(model, feature_names):
    def predict(X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=feature_names)
        return model.predict(X)
    return predict

# Feature-Namen
feature_names_bike = X_bike_proc_sampled.columns.tolist()
feature_names_cal = X_cal_proc_sampled.columns.tolist()

n_features_bike = X_bike_proc_sampled.shape[1]
n_features_cal = X_cal_proc_sampled.shape[1]

print(f"Number of features in bike dataset: {n_features_bike}")
print(f"Number of features in cal dataset: {n_features_cal}")


Bike Sharing RandomForestRegressor trainiert.
California Housing RandomForestRegressor trainiert.
Bike Sharing LinearRegression trainiert.
California Housing LinearRegression trainiert.
Number of features in bike dataset: 16
Number of features in cal dataset: 8


In [4]:
import shapiq.explainer

print(dir(shapiq.explainer))

['AgnosticExplainer', 'Explainer', 'TabPFNExplainer', 'TabularExplainer', 'TreeExplainer', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'agnostic', 'base', 'configuration', 'custom_types', 'tabpfn', 'tabular', 'tree', 'utils', 'validation']


In [5]:
import shapiq.explainer.tabular

print(dir(shapiq.explainer.tabular))

['Any', 'Explainer', 'ExplainerIndices', 'InteractionValues', 'Literal', 'TYPE_CHECKING', 'TabularExplainer', 'TabularExplainerApproximators', 'TabularExplainerImputers', 'TabularExplainerIndices', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'annotations', 'finalize_computed_interactions', 'overrides', 'setup_approximator', 'warn']


In [6]:
from shapiq.explainer.tabular import TabularExplainerApproximators

print(TabularExplainerApproximators)

typing.Literal['spex', 'montecarlo', 'svarm', 'permutation', 'regression']


In [7]:
# Hintergrunddaten
sample_size = 100
background_bike_df = X_bike_proc_sampled.sample(n=sample_size, random_state=42).reset_index(drop=True)
background_cal_df = X_cal_proc_sampled.sample(n=sample_size, random_state=42).reset_index(drop=True)

background_bike_np = background_bike_df.to_numpy()
background_cal_np = background_cal_df.to_numpy()

# Wrapped Predict Funktionen
wrapped_predict_bike_rf = model_predict_wrapper(model_bike_rf, feature_names_bike)
wrapped_predict_cal_rf = model_predict_wrapper(model_cal_rf, feature_names_cal)
wrapped_predict_bike_lr = model_predict_wrapper(model_bike_lr, feature_names_bike)
wrapped_predict_cal_lr = model_predict_wrapper(model_cal_lr, feature_names_cal)

# --- shapiq Explainer ---

# Bike RF
explainer_shapiq_spex_bike_rf = shapiq.TabularExplainer(wrapped_predict_bike_rf, background_bike_np, approximator="spex", sample_size=sample_size)
explainer_shapiq_svarm_bike_rf = shapiq.TabularExplainer(wrapped_predict_bike_rf, background_bike_np, approximator="svarm", sample_size=sample_size)
explainer_shapiq_perm_bike_rf = shapiq.TabularExplainer(wrapped_predict_bike_rf, background_bike_np, approximator="permutation", sample_size=sample_size)

# Cal RF
explainer_shapiq_spex_cal_rf = shapiq.TabularExplainer(wrapped_predict_cal_rf, background_cal_np, approximator="spex", sample_size=sample_size)
explainer_shapiq_svarm_cal_rf = shapiq.TabularExplainer(wrapped_predict_cal_rf, background_cal_np, approximator="svarm", sample_size=sample_size)
explainer_shapiq_perm_cal_rf = shapiq.TabularExplainer(wrapped_predict_cal_rf, background_cal_np, approximator="permutation", sample_size=sample_size)

# Bike LR
explainer_shapiq_spex_bike_lr = shapiq.TabularExplainer(wrapped_predict_bike_lr, background_bike_np, approximator="spex", sample_size=sample_size)
explainer_shapiq_svarm_bike_lr = shapiq.TabularExplainer(wrapped_predict_bike_lr, background_bike_np, approximator="svarm", sample_size=sample_size)
explainer_shapiq_perm_bike_lr = shapiq.TabularExplainer(wrapped_predict_bike_lr, background_bike_np, approximator="permutation", sample_size=sample_size)

# Cal LR
explainer_shapiq_spex_cal_lr = shapiq.TabularExplainer(wrapped_predict_cal_lr, background_cal_np, approximator="spex", sample_size=sample_size)
explainer_shapiq_svarm_cal_lr = shapiq.TabularExplainer(wrapped_predict_cal_lr, background_cal_np, approximator="svarm", sample_size=sample_size)
explainer_shapiq_perm_cal_lr = shapiq.TabularExplainer(wrapped_predict_cal_lr, background_cal_np, approximator="permutation", sample_size=sample_size)

# --- shap KernelExplainer (Referenz-Explainer) ---

explainer_shap_kernel_bike_rf = shap.KernelExplainer(wrapped_predict_bike_rf, background_bike_df, feature_names=feature_names_bike)
explainer_shap_kernel_cal_rf = shap.KernelExplainer(wrapped_predict_cal_rf, background_cal_df, feature_names=feature_names_cal)
explainer_shap_kernel_bike_lr = shap.KernelExplainer(wrapped_predict_bike_lr, background_bike_df, feature_names=feature_names_bike)
explainer_shap_kernel_cal_lr = shap.KernelExplainer(wrapped_predict_cal_lr, background_cal_df, feature_names=feature_names_cal)

print("Alle Explainer (shapiq + shap) erfolgreich initialisiert.")


Alle Explainer (shapiq + shap) erfolgreich initialisiert.


In [8]:
def l1_error(a, b):
    return np.mean(np.abs(a - b))

def l2_error(a, b):
    return np.mean((a - b) ** 2)

def benchmark_shap_explainers(explainers, X, ref_shap_values, budget=65536):
    import time
    import numpy as np
    results = []
    n_samples = X.shape[0]
    n_features = X.shape[1]

    for name, explainer in explainers.items():
        print(f"→ Benchmarking: {name}")
        try:
            start = time.time()
            shap_values = explainer.explain(X, budget=budget)
            duration = time.time() - start
            
            shap_values = np.array(shap_values)

            # Falls 1D-Array, auf (n_samples, n_features) reshapen
            if shap_values.ndim == 1 and shap_values.shape[0] == n_samples * n_features:
                shap_values = shap_values.reshape(n_samples, n_features)
            elif shap_values.ndim != 2:
                raise ValueError(f"Unerwartete Form der Shapley-Werte: {shap_values.shape}")

            # Vergleich mit Referenz: L2 Fehler
            l2_error = np.linalg.norm(shap_values - ref_shap_values)

            results.append({
                "method": name,
                "l2_error": l2_error,
                "duration_sec": duration,
            })
        except Exception as e:
            print(f"[Fehler bei {name}]: {e}")
            results.append({
                "method": name,
                "l2_error": np.nan,
                "duration_sec": np.nan
            })

    return pd.DataFrame(results)


shapley

In [9]:
def timeit(func, *args, **kwargs):
    import time
    start_time = time.time()
    result = func(*args, **kwargs)
    duration = time.time() - start_time
    return result, duration

ref_sample_size = 50

print("Berechne Referenz-Shapley-Werte (KernelExplainer)...")

ref_shap_bike_rf, _ = timeit(
    lambda: np.array(explainer_shap_kernel_bike_rf.shap_values(X_bike_proc_sampled.iloc[:ref_sample_size]))
)

ref_shap_cal_rf, _ = timeit(
    lambda: np.array(explainer_shap_kernel_cal_rf.shap_values(X_cal_proc_sampled.iloc[:ref_sample_size]))
)

ref_shap_bike_lr, _ = timeit(
    lambda: np.array(explainer_shap_kernel_bike_lr.shap_values(X_bike_proc_sampled.iloc[:ref_sample_size]))
)

ref_shap_cal_lr, _ = timeit(
    lambda: np.array(explainer_shap_kernel_cal_lr.shap_values(X_cal_proc_sampled.iloc[:ref_sample_size]))
)


Berechne Referenz-Shapley-Werte (KernelExplainer)...


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
def validate_shapes(ref_dict, n_expected, n_features_expected):
    print("Starte Validierung...")

    for name, arr in ref_dict.items():
        print(f"→ {name}")
        if arr is None:
            print("  Fehler: None")
            continue
        if isinstance(arr, list):
            arr = np.array(arr)
        if arr.dtype == "object":
            print("  Fehler: dtype=object")
        if np.isnan(arr).any():
            print("  Fehler: NaN enthalten")
        if len(arr.shape) != 2:
            print(f"  Fehler: Nicht 2D (shape={arr.shape})")
        elif arr.shape[0] != n_expected:
            print(f"  Warnung: {arr.shape[0]} Zeilen, erwartet: {n_expected}")
        elif arr.shape[1] != n_features_expected:
            print(f"  Warnung: {arr.shape[1]} Features, erwartet: {n_features_expected}")
        else:
            print(f"  OK: shape={arr.shape}, dtype={arr.dtype}")

validate_shapes(
    {
        "ref_shap_bike_rf": ref_shap_bike_rf,
        "ref_shap_bike_lr": ref_shap_bike_lr,
    },
    n_expected=ref_sample_size,
    n_features_expected=X_bike_proc_sampled.shape[1]
)

validate_shapes(
    {
        "ref_shap_cal_rf": ref_shap_cal_rf,
        "ref_shap_cal_lr": ref_shap_cal_lr,
    },
    n_expected=ref_sample_size,
    n_features_expected=X_cal_proc_sampled.shape[1]
)

# --- Explainer-Dictionaries ---
explainers_bike_rf = {
    "shapiq_spex": explainer_shapiq_spex_bike_rf,
    "shapiq_svarm": explainer_shapiq_svarm_bike_rf,
    "shapiq_perm": explainer_shapiq_perm_bike_rf,
}

explainers_cal_rf = {
    "shapiq_spex": explainer_shapiq_spex_cal_rf,
    "shapiq_svarm": explainer_shapiq_svarm_cal_rf,
    "shapiq_perm": explainer_shapiq_perm_cal_rf,
}

explainers_bike_lr = {
    "shapiq_spex": explainer_shapiq_spex_bike_lr,
    "shapiq_svarm": explainer_shapiq_svarm_bike_lr,
    "shapiq_perm": explainer_shapiq_perm_bike_lr,
}

explainers_cal_lr = {
    "shapiq_spex": explainer_shapiq_spex_cal_lr,
    "shapiq_svarm": explainer_shapiq_svarm_cal_lr,
    "shapiq_perm": explainer_shapiq_perm_cal_lr,
}

# --- Benchmark ---
print("Benchmark: Bike Sharing - RandomForest")
results_bike_rf = benchmark_shap_explainers(explainers_bike_rf, X_bike_proc_sampled, ref_shap_bike_rf)

print("Benchmark: California Housing - RandomForest")
results_cal_rf = benchmark_shap_explainers(explainers_cal_rf, X_cal_proc_sampled, ref_shap_cal_rf)

print("Benchmark: Bike Sharing - LinearRegression")
results_bike_lr = benchmark_shap_explainers(explainers_bike_lr, X_bike_proc_sampled, ref_shap_bike_lr)

print("Benchmark: California Housing - LinearRegression")
results_cal_lr = benchmark_shap_explainers(explainers_cal_lr, X_cal_proc_sampled, ref_shap_cal_lr)

results_dfs = [
    df.assign(method=lambda d: d["method"] + " | Bike-RF") for df in [results_bike_rf] if not df.empty
] + [
    df.assign(method=lambda d: d["method"] + " | Bike-LR") for df in [results_bike_lr] if not df.empty
] + [
    df.assign(method=lambda d: d["method"] + " | Cal-RF") for df in [results_cal_rf] if not df.empty
] + [
    df.assign(method=lambda d: d["method"] + " | Cal-LR") for df in [results_cal_lr] if not df.empty
]

df_results = pd.concat(results_dfs, ignore_index=True)

# --- Zusammenführen ---
results_list = []

if not results_bike_rf.empty:
    results_list.append(results_bike_rf.assign(method=lambda df: df["method"] + " | Bike-RF"))
if not results_bike_lr.empty:
    results_list.append(results_bike_lr.assign(method=lambda df: df["method"] + " | Bike-LR"))
if not results_cal_rf.empty:
    results_list.append(results_cal_rf.assign(method=lambda df: df["method"] + " | Cal-RF"))
if not results_cal_lr.empty:
    results_list.append(results_cal_lr.assign(method=lambda df: df["method"] + " | Cal-LR"))

df_results = pd.concat(results_list, ignore_index=True)

print("Ergebnisse (Beispiel, SVARM):")
display(df_results[df_results["method"].str.contains("svarm")].head(1))

print("Benchmark abgeschlossen.")


Starte Validierung...
→ ref_shap_bike_rf
  OK: shape=(50, 16), dtype=float64
→ ref_shap_bike_lr
  OK: shape=(50, 16), dtype=float64
Starte Validierung...
→ ref_shap_cal_rf
  OK: shape=(50, 8), dtype=float64
→ ref_shap_cal_lr
  OK: shape=(50, 8), dtype=float64
Benchmark: Bike Sharing - RandomForest
→ Benchmarking: shapiq_spex


Ergebnisse in DataFrame

Visualisierung (Plotting)

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(data=df_results, x="method", y="runtime", errorbar='sd')
plt.xticks(rotation=45, ha="right")
plt.title("Durchschnittliche Laufzeit je Methode und Datensatz")
plt.ylabel("Laufzeit (Sekunden)")
plt.xlabel("Methode | Datensatz")
plt.tight_layout()
plt.show()

plt.figure(figsize=(14,6))
sns.boxplot(data=df_results, x="method", y="quality")
plt.xticks(rotation=45, ha="right")
plt.title("Qualitätsfehler (L2-Abweichung) je Methode und Datensatz")
plt.ylabel("L2-Abweichung zum Referenz")
plt.xlabel("Methode | Datensatz")
plt.tight_layout()
plt.show()

plt.figure(figsize=(16,8))
sns.violinplot(data=df_results, x="method", y="runtime", inner="quartile", color="skyblue")
plt.xticks(rotation=45, ha="right")
plt.title("Verteilung der Laufzeiten je Methode und Datensatz")
plt.ylabel("Laufzeit (Sekunden)")
plt.xlabel("Methode | Datensatz")
plt.tight_layout()
plt.show()

plt.figure(figsize=(16,8))
sns.violinplot(data=df_results, x="method", y="quality", inner="quartile", color="lightcoral")
plt.xticks(rotation=45, ha="right")
plt.title("Verteilung der Qualitätsfehler (L2) je Methode und Datensatz")
plt.ylabel("L2-Abweichung zum Referenz")
plt.xlabel("Methode | Datensatz")
plt.tight_layout()
plt.show()
